In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import networkx as nx
import matplotlib.pyplot as plt

# Step 1: Load the data
data = pd.read_csv('data/Macroeconomic_data.csv')  # Replace with your actual file name

# Step 2: Data Preprocessing
data.fillna(method='ffill', inplace=True)


FileNotFoundError: [Errno 2] No such file or directory: 'data/Macroeconomic_data.csv'

In [4]:

# Step 3: Feature Engineering
features = ['Net trade in goods and services (BoP, current US$)',
            'GDP (current US$)',
            'Consumer price index (2010 = 100)',
            'Unemployment, total (% of total labor force) (national estimate)',
            'Exports of goods and services (BoP, current US$)',
            'Imports of goods and services (BoP, current US$)',
            'Foreign direct investment, net (BoP, current US$)',
            'Official exchange rate (LCU per US$, period average)']

# Target is the next year's exchange rate, so we shift the target column by -1
data['Next Year Exchange Rate'] = data['Official exchange rate (LCU per US$, period average)'].shift(-1)

# Drop the last row since it doesn't have a corresponding next year exchange rate
data.dropna(inplace=True)

In [30]:
# Step 4: Graph Construction with Edge Weights of 1 for All Edges

df = pd.DataFrame(data)
unique_countries = []

# Create a mapping from country code to index
for col in df.columns:
    country_codes = str(df[col].iloc[0])
    if country_codes not in unique_countries and len(country_codes) == 3:
        unique_countries.append(country_codes)
    country_to_index = {code: i for i, code in enumerate(unique_countries)}

    

# Create node features
node_features_list = []
for country in unique_countries:
    country_data = [unique_countries == country]
    node_features_list.append(country_data[features].values)

node_features = np.array(node_features_list)
node_features = torch.tensor(node_features, dtype=torch.float).squeeze(1)

# Create edge index and set all edge weights to 1
edge_index = []
edge_attr = []

# Create a fully connected graph (or some other structure) with all edges having weight 1
for i in range(len(unique_countries)):
    for j in range(i + 1, len(unique_countries)):
        edge_index.append([i, j])
        edge_index.append([j, i])  # Assuming undirected graph
        edge_attr.append(1.0)
        edge_attr.append(1.0)

# Convert to PyTorch tensors
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
edge_attr = torch.tensor(edge_attr, dtype=torch.float)

# Labels (next year's exchange rate for each country)
labels = []
for country in unique_countries:
    country_data = [unique_countries == country]
    labels.append(country_data['Next Year Exchange Rate'].values[0])

labels = torch.tensor(labels, dtype=torch.float)

# Normalize node features
scaler = StandardScaler()
node_features = torch.tensor(scaler.fit_transform(node_features), dtype=torch.float)

# Create PyTorch Geometric data object
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=labels)


KeyError: 'Country Code'

In [ ]:
# Step 5: Define the KA-GCN Model
class KAGCN(nn.Module):
    def __init__(self, num_node_features, hidden_dim, output_dim, dropout=0.3):
        super(KAGCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index, edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        return x





In [ ]:
# Step 6: Training the Model
def train_model(model, data, labels, num_epochs=500, learning_rate=0.01):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        
        outputs = model(data.x, data.edge_index, data.edge_attr)
        loss = criterion(outputs.squeeze(), labels)
        
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Initialize and train the model
model = KAGCN(num_node_features=node_features.shape[1], hidden_dim=64, output_dim=1)
train_model(model, data, labels)

In [ ]:
# Step 7: Prediction
model.eval()
with torch.no_grad():
    predictions = model(data.x, data.edge_index, data.edge_attr)

In [ ]:
# Step 8: Evaluate Predictions
mse = mean_squared_error(labels.numpy(), predictions.numpy())
print(f'Test MSE: {mse}')

# Example prediction for the next year
for i, country in enumerate(country_codes):
    print(f"Country: {country}, Predicted Exchange Rate for Next Year: {predictions[i].item()}")

In [ ]:
# Step 9: Visualize the Graph
def visualize_graph(edge_index, node_labels):
    G = nx.Graph()
    
    # Add nodes with labels
    for i, label in enumerate(node_labels):
        G.add_node(i, label=label)
    
    # Add edges
    edge_list = edge_index.t().tolist()  # Convert to list of edge pairs
    G.add_edges_from(edge_list)
    
    # Draw the graph
    pos = nx.spring_layout(G)  # Layout for visualization
    nx.draw(G, pos, with_labels=True, labels={i: label for i, label in enumerate(node_labels)}, node_size=500, font_size=10)
    plt.show()

# Visualize the graph
visualize_graph(edge_index, country_codes)